In [1]:
import pandas as pd
import numpy as np
import dropbox
import os
import matplotlib.pyplot as plt
# import altair as alt
# alt.renderers.enable('notebook')

In [2]:
# class Point():
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y

#     def __repr__(self):
#         return 'Point({0},{1})'.format(self.x,self.y)

#     def translate(self, dx, dy):
#         "Add dx and dy to the point."
#         self.x += dx
#         self.y += dy

# e = Point(1.5, 2.5)

In [2]:
dbx = dropbox.Dropbox(os.environ['DROPBOX_API'])
dbx.users_get_current_account()

FullAccount(account_id='dbid:AABT1H-NJgK-ns_10QubSjssbwKMHtznqDU', name=Name(given_name='saurav', surname='chowdhury', familiar_name='saurav', display_name='saurav chowdhury', abbreviated_name='SC'), email='saurav13iitr@gmail.com', email_verified=True, disabled=False, locale='en', referral_link='https://www.dropbox.com/referrals/AADGzvUD1naruTWe92WGn9EaGcI-C2M2Zfg?src=app9-7501328', is_paired=False, account_type=AccountType('basic', None), root_info=UserRootInfo(root_namespace_id='884484377', home_namespace_id='884484377'), profile_photo_url=None, country='IN', team=None, team_member_id=None)

In [3]:
years = list(range(2002, 2020))
final_df = pd.DataFrame()

for year in years:
    path= "/Saurav/data/Stage_Specific_Ranking_-_Table_data"
    _, res = dbx.files_download(path+str(year)+".csv")
    df=pd.read_csv(res.raw)
    final_df = pd.concat([df,final_df])

final_df.shape

(21726, 114)

In [4]:
len(final_df['Athlete/Comp'].unique()) ## Unique ID for athele => Data at Athlete X Country X Competetion X Round lvl

21726

In [5]:
filtered_data = final_df.loc[:, ['Athlete/Comp','Metric', 'Score','Finish','Gender','Judge 1 Score','Judge 2 Score',
                          'Judge 3 Score','Judge 4 Score','Judge 5 Score','Judge 6 Score - 1','Judge 7 Score - 1',
                          'Round Group', 'Season','Season/Comp','Round','Wind']]

# filtered_data['age_yrs'] = 2020 - final_df['yob']
filtered_data['Athlete_name'] =  final_df['Athlete/Comp'].str.extract(r"(.*-....).-", expand=False)

filtered_data = filtered_data.reset_index(drop = True)

In [6]:
filtered_data.columns = filtered_data.columns.str.replace("/","_").str.replace("\ -\ ","_").str.replace("\ ","_")

In [7]:
filtered_data.columns

Index(['Athlete_Comp', 'Metric', 'Score', 'Finish', 'Gender', 'Judge_1_Score',
       'Judge_2_Score', 'Judge_3_Score', 'Judge_4_Score', 'Judge_5_Score',
       'Judge_6_Score_1', 'Judge_7_Score_1', 'Round_Group', 'Season',
       'Season_Comp', 'Round', 'Wind', 'Athlete_name'],
      dtype='object')

In [8]:
filtered_data[filtered_data['Metric'] != filtered_data['Score']].loc[:, 'Season'].unique()

array([2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004,
       2003, 2002], dtype=int64)

### Updating old scores in same scale as new metric


In [9]:

filtered_data['Metric'] = [round(filtered_data.loc[i,'Score']/30*100, 2) if filtered_data.loc[i, 'Metric'] != filtered_data.loc[i,'Score'] \
        else filtered_data.loc[i, 'Metric'] for i in range(len(filtered_data))]

filtered_data[filtered_data['Season'] == 2010].head()

,Athlete_Comp,Metric,Score,Finish,Gender,Judge_1_Score,Judge_2_Score,Judge_3_Score,Judge_4_Score,Judge_5_Score,Judge_6_Score_1,Judge_7_Score_1,Round_Group,Season,Season_Comp,Round,Wind,Athlete_name
13699,SMYSHLIAEV Aleksandr - OAR - World Cup 5 - 201...,52.77,15.83,15,Men,2.5,2.5,1.1,2.7,3.3,2.0,2.0,Finals,2010,2010-World Cup 5,Final 1,NaN,SMYSHLIAEV Aleksandr - OAR
13700,WILSON Bryon - USA - World Cup 7 - 2010 - Final 1,81.00,24.30,8,Men,4.3,4.6,4.5,4.5,4.4,2.0,2.0,Finals,2010,2010-World Cup 7,Final 1,0.0,WILSON Bryon - USA
13701,DUFOUR LAPOINTE Justine - CAN - North American...,68.87,20.66,6,Women,3.6,4.0,3.9,4.0,3.7,14.0,14.0,Qualification,2010,2010-North American Cup 2,Qualification 1,NaN,DUFOUR LAPOINTE Justine - CAN
13702,SALONEN Janne - FIN - World Cup 2 - 2010 - Qua...,69.20,20.76,39,Men,3.8,3.9,3.6,3.7,3.9,2.5,2.3,Qualification,2010,2010-World Cup 2,Qualification,4.0,SALONEN Janne - FIN
13703,KEATING Nicholas - USA - North American Cup 2 ...,69.87,20.96,19,Men,3.6,3.6,3.7,3.7,4.0,20.0,17.0,Qualification,2010,2010-North American Cup 2,Qualification 1,NaN,KEATING Nicholas - USA


### Adding rankings upto 30 for each competition

In [10]:
# Dropping the scores column
filtered_data = filtered_data.drop(columns = ['Score'])

#filtering the finals only
# filtered_data = filtered_data[filtered_data['Round'].str.lower().str.extract(r'.*(final).*', expand=False).notnull()]

print(filtered_data.shape)
#dividing competition data into 3 levels
tier1_df = filtered_data[filtered_data['Season_Comp'].str.lower().str.extract(r'.*(world).*', expand=False).notnull() |\
                 filtered_data['Season_Comp'].str.lower().str.extract(r'.*(olympics).*', expand=False).notnull()]

print(tier1_df.shape)

tier2_df = filtered_data[filtered_data['Season_Comp'].str.lower().str.extract(r'.*(north american).*', expand=False).notnull()]
print(tier2_df.shape)

tier3_df = filtered_data[filtered_data['Season_Comp'].str.lower().str.extract(r'.*(canad).*', expand=False).notnull()]
print(tier3_df.shape)

print(tier1_df.shape[0]+tier2_df.shape[0]+tier3_df.shape[0])

(21726, 17)
(15890, 17)
(4585, 17)
(1251, 17)
21726


In [19]:
# Dividing into men and women
tier1_df_M = tier1_df[tier1_df['Gender'] == "Men"]
tier1_df_W = tier1_df[tier1_df['Gender'] == "Women"]
tier2_df_M = tier2_df[tier2_df['Gender'] == "Men"]
tier2_df_W = tier2_df[tier2_df['Gender'] == "Women"]
tier3_df_M = tier3_df[tier3_df['Gender'] == "Men"]
tier3_df_W = tier3_df[tier3_df['Gender'] == "Women"]


In [20]:
years = list(range(2002, 2020))
for year in years:
    comps = tier1_df_M[(tier1_df_M['Season'] == year)].loc[:, 'Season_Comp'].unique() 
    for comp in comps:
        print(comp)
        rounds = tier1_df_M[(tier1_df_M['Season'] == year) & (tier1_df_M['Season_Comp'] == comp)].loc[:, 'Round'].unique()
        print(sorted(rounds))

2002-Olympics
['Final 1', 'Qualification 1']
2003-World Cup 10
['Final 1', 'Qualification']
2003-World Cup 8
['Final 1', 'Qualification']
2003-World Cup 5
['Final 1', 'Qualification']
2003-World Cup 6
['Final 1', 'Qualification']
2003-World Cup 4
['Final 1', 'Qualification']
2003-World Cup 7
['Final 1', 'Qualification']
2003-World Cup 2
['Final 1', 'Qualification']
2003-World Cup 1
['Final 1', 'Qualification']
2003-World Championships
['Final', 'Qualification']
2003-World Cup 3
['Final 1', 'Qualification']
2003-World Cup 9
['Final 1', 'Qualification']
2004-World Cup 3
['Final 1', 'Qualification']
2004-World Cup 2
['Final 1', 'Qualification']
2004-World Cup 9
['Final 1', 'Qualification']
2004-World Cup 7
['Final 1', 'Qualification']
2004-World Cup 1
['Final 1', 'Qualification']
2004-World Cup 6
['Final 1', 'Qualification']
2004-World Cup 4
['Final 1', 'Qualification']
2004-World Cup 8
['Final 1', 'Qualification']
2004-World Cup 10
['Final 1', 'Qualification']
2004-World Cup 5
['Final 1'

In [15]:
tier3_df['Round_Group'].unique()

array(['Qualification', 'Finals'], dtype=object)

In [61]:
# Function to rank all players

year = 2008
comp = "2008-World Cup 7"


def rank_30_athletes(year, comp):
    rounds = ['Qualification', 'Finals']
    
    temp_df = tier1_df_M[(tier1_df_M['Season'] == year) & (tier1_df_M['Season_Comp'] == comp)]
    
    print(temp_df.loc[:, ['Finish', 'Metric']].sort_values('Finish'))
    q_rounds = temp_df[(temp_df['Round_Group'] == rounds[0])].loc[:, 'Round'].unique()
    q_rounds = sorted(q_rounds, reverse = True)
    
    f_rounds = temp_df[(temp_df['Round_Group'] == rounds[1])].loc[:, 'Round'].unique()
    f_rounds = sorted(f_rounds, reverse = True)
    
    # sorting the rounds in descending order of importance e.g final 1, qualifier 2, qualifier 1, ..
    rounds_desc_order = f_rounds + q_rounds
    
    output_df = temp_df[(temp_df['Round']== rounds_desc_order[0])].sort_values('Finish')
    max_rank = max(output_df['Finish'])
    min_score = min(output_df['Metric'])
    prev_round = 1 #assuming there are more than 1 round
    
    while max_rank < 30:
        
        if len(rounds_desc_order) <= prev_round: # checking if the previous round exists in the competition
            break
        
        # creating the dataframe for the rounds prior to the final
        next_ranks_df = temp_df[(temp_df['Round']== rounds_desc_order[prev_round])].sort_values('Finish')
        
        score_adj = next_ranks_df[next_ranks_df['Finish']== max_rank]['Metric'].values - \
                    next_ranks_df[next_ranks_df['Finish']== max_rank+1]['Metric'].values
        
        # adjusting scores for the next rounds
        new_score = min_score - score_adj
        
        # Filtering out the ranks already covered in the output dataframe
        next_ranks_df = next_ranks_df[next_ranks_df['Finish']> max_rank]

        for rank in next_ranks_df['Finish']:

            score_adj = next_ranks_df[next_ranks_df['Finish']== rank]['Metric'].values - \
                        next_ranks_df[next_ranks_df['Finish']== rank+1]['Metric'].values

            next_ranks_df.loc[next_ranks_df['Finish'] == rank, 'Metric'] = new_score

            new_score = new_score - score_adj
        
        #adding the new ranks with adjusted scores to the output
        output_df = pd.concat([output_df, next_ranks_df], axis = 0, ignore_index = True)
        max_rank = max(output_df['Finish'])
        
        if max_rank > 30:
            output_df = output_df[output_df['Finish'] <=30]
        else:
            prev_round = prev_round + 1
            
    return output_df
            
out = rank_30_athletes(year, comp)

print(out.loc[:, ['Finish', 'Metric']])

       Finish  Metric
16596       1   77.53
16246       2   77.47
16187       3   77.27
16588       4   77.07
16436       5   76.37
16683       6   76.20
16653       7   75.17
16216       8   72.87
16174       9   72.83
16385      10   72.73
16337      11   69.73
16695      12   68.90
16042      13   68.53
16043      14   68.10
16145      15   64.83
16492      16   56.07
16679      17   51.07
16153      18   45.17
16467      19   35.30
16603      20    9.57
       Finish  Metric
16596       1   77.53
16246       2   77.47
16187       3   77.27
16588       4   77.07
16436       5   76.37
16683       6   76.20
16653       7   75.17
16216       8   72.87
16174       9   72.83
16385      10   72.73
16337      11   69.73
16695      12   68.90
16042      13   68.53
16043      14   68.10
16145      15   64.83
16492      16   56.07
16679      17   51.07
16153      18   45.17
16467      19   35.30
16603      20    9.57


In [67]:
tier1_df = tier1_df.loc[:, ['Athlete_Comp', 'Athlete_name', 'Gender', 'age_yrs', 'Nation', 'Judge_1_Score', 'Judge_2_Score', 
                    'Judge_3_Score', 'Judge_4_Score', 'Judge_5_Score', 'Judge_6_Score_1', 'Judge_7_Score_1', 'Major_Finalist',
                    'Major_Medalist', 'Olympic_Finalist', 'Olympic_Medalist','Temp_Snow', 'Temp_Air','Wind',
                    'Season', 'Season_Comp', 'Round', 'Metric', 'Finish']]

tier2_df= tier2_df.loc[:, ['Athlete_Comp', 'Athlete_name', 'Gender', 'age_yrs', 'Nation', 'Judge_1_Score', 'Judge_2_Score', 
                    'Judge_3_Score', 'Judge_4_Score', 'Judge_5_Score', 'Judge_6_Score_1', 'Judge_7_Score_1', 'Major_Finalist',
                    'Major_Medalist', 'Olympic_Finalist', 'Olympic_Medalist','Temp_Snow', 'Temp_Air','Wind',
                    'Season', 'Season_Comp', 'Round', 'Metric', 'Finish']]

tier3_df= tier3_df.loc[:, ['Athlete_Comp', 'Athlete_name', 'Gender', 'age_yrs', 'Nation', 'Judge_1_Score', 'Judge_2_Score', 
                    'Judge_3_Score', 'Judge_4_Score', 'Judge_5_Score', 'Judge_6_Score_1', 'Judge_7_Score_1', 'Major_Finalist',
                    'Major_Medalist', 'Olympic_Finalist', 'Olympic_Medalist','Temp_Snow', 'Temp_Air','Wind',
                    'Season', 'Season_Comp', 'Round', 'Metric', 'Finish']]

tier1_df = tier1_df.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})
tier2_df = tier2_df.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})
tier3_df = tier3_df.rename(columns = {'Athlete_Comp' : 'Athlete_Comp_ID', 'Finish' : 'Finish_Rank', 'Metric' : 'Score_Metric'})

In [68]:
tier3_df['Gender'].unique()

array(['Men', 'Women'], dtype=object)

In [70]:
tier1_df_M.to_csv("../data/tier_1_comp_data_men.csv", index = False)
tier2_df_M.to_csv("../data/tier_2_comp_data_men.csv", index = False)
tier3_df_M.to_csv("../data/tier_3_comp_data_men.csv", index = False)

tier1_df_W.to_csv("../data/tier_1_comp_data_women.csv", index = False)
tier2_df_W.to_csv("../data/tier_2_comp_data_women.csv", index = False)
tier3_df_W.to_csv("../data/tier_3_comp_data_women.csv", index = False)

In [75]:
files = ['tier_1_comp_data_men.csv', 'tier_2_comp_data_men.csv', 'tier_3_comp_data_men.csv',
         'tier_1_comp_data_women.csv', 'tier_2_comp_data_women.csv', 'tier_3_comp_data_women.csv']

local_path = "../data/"
dest_path= "/Saurav/data/compiled_data/"

for file in files:
    filepath = local_path+file
    
    with open(filepath, "rb") as f:
       # we want to overwite any previous version of the file
        try:
            dbx.files_upload(f.read(), dest_path+file, mode=dropbox.files.WriteMode("overwrite"))
            print(file+" uploaded")
        except ApiError as err:
            # This checks for the specific error where a user doesn't have enough Dropbox space quota to upload this file
            if (err.error.is_path() and
                    err.error.get_path().error.is_insufficient_space()):
                sys.exit("ERROR: Cannot back up; insufficient space.")
            elif err.user_message_text:
                print(err.user_message_text)
                sys.exit()
            else:
                print(err)
                sys.exit()


tier_1_comp_data_men.csv uploaded
tier_2_comp_data_men.csv uploaded
tier_3_comp_data_men.csv uploaded
tier_1_comp_data_women.csv uploaded
tier_2_comp_data_women.csv uploaded
tier_3_comp_data_women.csv uploaded
